In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def read_in_file(files):
  text_names = []
  for file in files:
    if re.match(r'\w*.txt', file):
      file = re.split(r'.txt', file)[0]
      text_names.append(file)
  return(text_names)


In [ ]:
#указываем все нужные директории
import re
TRAIN_DIR = '/content/drive/My Drive/terms/scienceie2017_train/train2' #данные для обучения
DEV_DIR = '/content/drive/My Drive/terms/scienceie2017_dev/dev' #данные для валидации
TEST_DIR = '/content/drive/My Drive/terms/semeval_articles_test' #данные для теста
SOURCE_DIR = '/content/drive/My Drive/terms/files/simple/basic' #для записи файлов в новом формате
DIR = '/content/drive/My Drive/terms/NER_Example/CONLL-2002'
GAZ_DIR = '/content/drive/My Drive/terms/gazetteer'

In [ ]:
#считываем названия файлов
train_files = sorted(os.listdir(TRAIN_DIR)) #считываем все названия файлов для обучения с расширениями
train_text_names = read_in_file(train_files) #список тестовых названий без расширения

dev_files = sorted(os.listdir(DEV_DIR)) #считываем все названия валидационных файлов с расширениями
dev_text_names = read_in_file(dev_files)

test_files = sorted(os.listdir(TEST_DIR)) #считываем все названия тестовых файлов с расширениями
test_text_names = read_in_file(test_files)



In [ ]:
print(train_text_names[-1])

S2352179114200056


 ## Считываем содержимое файлов

In [ ]:
def read_anns(text_names, dir):
  anns_all = {} #аннотации хранятся в виде словаря, где ключ - название документа, значение - словарь с ключом (begin, end, термин) и значением [тип]
  for text_name in text_names:
    ann_file = os.path.join(dir, text_name+'.ann')
    with open(ann_file, "r", encoding="utf8") as fin:
      anns = {}
      for line in fin:
        line = line.strip()
        line = line.split('\t')
        if len(line) == 3:
          term_type = line[1].split() 
          if (int(term_type[1]), int(term_type[-1]), line[2]) in anns.keys():
            continue
          else:
            anns[(int(term_type[1]), int(term_type[-1]), line[2])] = [term_type[0]]
      keys_ = anns.keys()
      keys = [key for key in keys_]
      for key in keys:
        l = key
        start = key[0]
        end = key[1]
        #keys.remove(key)
        for k in keys:
          if (k[0] == start and k[1] < end) or (k[0] > start and k[1] == end) or (k[0] > start and k[1] < end):
            keys.remove(k)
      anns_new = {}
      for key in keys:
        anns_new[key] = anns[key]
      anns_all[text_name] = anns_new
  return(anns_all)


In [ ]:
train_anns = read_anns(train_text_names, TRAIN_DIR)
dev_anns = read_anns(dev_text_names, DEV_DIR)
test_anns = read_anns(test_text_names, TEST_DIR)

In [ ]:
print(test_anns['S221450951400014X'])

{(56, 63, 'bitumen'): ['Material'], (158, 176, 'waste glass cullet'): ['Material'], (106, 123, 'stiffness modulus'): ['Process'], (334, 351, 'stiffness modulus'): ['Process'], (462, 480, 'waste glass cullet'): ['Material'], (589, 607, 'waste glass cullet'): ['Material'], (838, 852, 'Marshall tests'): ['Process'], (688, 707, 'bituminous material'): ['Material'], (952, 970, 'waste glass cullet'): ['Material'], (781, 793, 'glass cullet'): ['Material'], (1047, 1065, 'waste glass cullet'): ['Material'], (1120, 1138, 'waste glass cullet'): ['Material'], (1243, 1255, 'interlocking'): ['Process'], (1144, 1161, 'stiffness modulus'): ['Process'], (1313, 1325, 'glass cullet'): ['Material'], (1395, 1411, 'glass particles’'): ['Material'], (1476, 1493, 'stiffness modulus'): ['Process'], (1528, 1546, 'waste glass cullet'): ['Material'], (1682, 1699, 'stiffness modulus'): ['Process'], (1742, 1754, 'glass cullet'): ['Material'], (1865, 1883, 'waste glass cullet'): ['Material'], (127, 144, 'asphaltic s

In [ ]:
#устанваливаем пакеты для обработки 
%pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from nltk import pos_tag, word_tokenize
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def change_start(span, count):
  span_new = []
  for each in span:
    start = each[0] + count
    end = each[1] + count
    span_new.append((start,end))
  return span_new

In [ ]:
def text_tokenization(text_names, dir):
  texts_all = {} #тексты хранятся в виде словаря, где ключ - название документа, значение - список из [слово, (start,end)]
  for text_name in text_names:
    count = 0
    text_file = os.path.join(dir, text_name+'.txt')
    with open(text_file, "r", encoding="utf8") as fin:
      for line in fin:
        text_each = []
        line = line.strip()
        sents = nltk.sent_tokenize(line)
        for sent in sents:
          span = list(TreebankWordTokenizer().span_tokenize(sent)) #(start, end) токена
          span_new = change_start(span, count)
          word = TreebankWordTokenizer().tokenize(sent) #токенизация
          #pos = nltk.pos_tag(word_tokenize(sent))
          pos = pos_tag(word)
          words = [list(tup) for tup in zip(word, span_new, pos)]
          text_each.append(words)
          count = words[-1][1][1] + 1
      texts_all[text_name] = text_each
  return texts_all

In [ ]:
train_texts = text_tokenization(train_text_names, TRAIN_DIR)
dev_texts = text_tokenization(dev_text_names, DEV_DIR)
test_texts = text_tokenization(test_text_names, TEST_DIR)

In [ ]:
print(test_texts['S221450951400014X'])

[[['This', (0, 4), ('This', 'DT')], ['figure', (5, 11), ('figure', 'NN')], ['demonstrates', (12, 24), ('demonstrates', 'VBZ')], ['that', (25, 29), ('that', 'IN')], ['changes', (30, 37), ('changes', 'NNS')], ['in', (38, 40), ('in', 'IN')], ['the', (41, 44), ('the', 'DT')], ['measure', (45, 52), ('measure', 'NN')], ['of', (53, 55), ('of', 'IN')], ['bitumen', (56, 63), ('bitumen', 'NNS')], ['content', (64, 71), ('content', 'JJ')], ['create', (72, 78), ('create', 'VBP')], ['sizable', (79, 86), ('sizable', 'JJ')], ['differences', (87, 98), ('differences', 'NNS')], ['in', (99, 101), ('in', 'IN')], ['the', (102, 105), ('the', 'DT')], ['stiffness', (106, 115), ('stiffness', 'JJ')], ['modulus', (116, 123), ('modulus', 'NN')], ['of', (124, 126), ('of', 'IN')], ['asphaltic', (127, 136), ('asphaltic', 'JJ')], ['samples', (137, 144), ('samples', 'NNS')], ['that', (145, 149), ('that', 'WDT')], ['include', (150, 157), ('include', 'VBP')], ['waste', (158, 163), ('waste', 'NN')], ['glass', (164, 169), 

 Сохраняем тексты для дальнейшего измерения качества

In [ ]:
def read_raw_text(text_names):
  raw_texts = {}
  for text in text_names:
    text_file = os.path.join(TEST_DIR, text+'.txt')
    with open(text_file, "r", encoding="utf8") as fin:
      text_raw = fin.read()
    raw_texts[text] = text_raw
  return raw_texts

In [ ]:
test_raw_texts = read_raw_text(test_text_names)
print(test_raw_texts['S221450951400014X'])

This figure demonstrates that changes in the measure of bitumen content create sizable differences in the stiffness modulus of asphaltic samples that include waste glass cullet. As the percentage of glass increases, the measure of the stiffness modulus of modified asphalt increases too. But with pass of optimum measure of glass the stiffness modulus of asphaltic samples decrease. This trend in total of percentages of bitumen content is existing. Due to that waste glass cullet has no suction; the trend does not extend to measuring the stiffness modulus of asphaltic samples including waste glass cullet with different percentage of bitumen content. Glass particles do not absorb any bituminous material, so it is necessary to decrease the bitumen content with the addition of glass cullet. According to Fig. 2 and the results of the Marshall tests, the optimum bitumen measures decrease significantly in samples that include higher percentages of waste glass cullet. As the percentage of optimum

## Создание газетира


In [ ]:
def make_gaz(dir, name):
  file = os.path.join(dir, name)
  terms = []
  count = 0
  with open(file, "r", encoding="utf8") as fin:
    for line in fin:
      count += 1
      line = line.strip()
      term = line.split(' ')
      for each in term:
        if each in stop_words: #убираем стоп-слова из газетира
          continue
        terms.append(each.lower())  
  print(count)
  return terms


In [ ]:
gaz_mat = make_gaz(GAZ_DIR, 'ms.txt')
gaz_comp = make_gaz(GAZ_DIR, 'cs.txt')
gaz_phys = make_gaz(GAZ_DIR, 'physics.txt')
gaz_plas = make_gaz(GAZ_DIR, 'pm.txt')

In [ ]:
gazetteers = [gaz_mat,gaz_comp,gaz_phys,gaz_plas]
for gaz in gazetteers:
  print(gaz)
  print(len(gaz))

Для каждого слова ставим метку, если он термин - тип-B/I/E, иначе - O

In [ ]:
def annotator(text, word, keys, anns):
  for key in keys:
    metric=['O']
    if word[1][0] == key[0]:
      if word[1][1] == key[1]:
        metric = []
        metric.append(anns[text][key])
        metric.append('B')
        break
      if word[1][1] < key[1]:
        metric = []
        metric.append(anns[text][key])
        metric.append('B')
        break
    elif word[1][0] > key[0] and word[1][1] <= key[1]:
      metric = []
      metric.append(anns[text][key])
      metric.append('I')
      break
  return metric


In [ ]:
def anns_out(text_names, texts, anns, dir, out):
  with open(os.path.join(dir, out), 'w', encoding='utf8') as fout:
    for text_name in text_names:
      for sent in texts[text_name]:
        for word in sent:
          #if word[0] in stop_words: #удаление частотных слов
            #continue
          gaz_vec = []
          for gaz in gazetteers:
            if word[0].lower() in gaz:
              gaz_vec.append(1)
            else:
              gaz_vec.append(0)
          if word[0] in stop_words:
            gaz_vec = [0, 0, 0, 0]
          metrics = ''
          metric = annotator(text_name, word, anns[text_name], anns)
          if len(metric) == 2:
            metrics = metric[1]+'-'+metric[0][0]
          else:
            metrics = metric[0]
          fout.write('{} {} {} {} {} {}\n'.format(word[0],gaz_vec[0],gaz_vec[1],gaz_vec[2],gaz_vec[3],metrics))
        fout.write('\n')

In [ ]:
train = anns_out(train_text_names, train_texts, train_anns, SOURCE_DIR, 'term.train')
dev = anns_out(dev_text_names, dev_texts, dev_anns, SOURCE_DIR, 'term.testa')
test = anns_out(test_text_names, test_texts, test_anns, SOURCE_DIR, 'term.testb')

## Обучение модели

In [ ]:
!pip install git+https://github.com/deepmipt/deeppavlov.git@sorokin/single_pass_elmo
#!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu_with_features
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu
!pip install tensorflow==1.14.0
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.4


  Cloning https://github.com/deepmipt/deeppavlov.git (to revision sorokin/single_pass_elmo) to /tmp/pip-req-build-63z035k9
  Running command git clone -q https://github.com/deepmipt/deeppavlov.git /tmp/pip-req-build-63z035k9
  Running command git checkout -b sorokin/single_pass_elmo --track origin/sorokin/single_pass_elmo
  Switched to a new branch 'sorokin/single_pass_elmo'
  Branch 'sorokin/single_pass_elmo' set up to track remote branch 'sorokin/single_pass_elmo' from 'origin'.
  Created wheel for deeppavlov: filename=deeppavlov-0.8.0-cp36-none-any.whl size=774394 sha256=4524617ad2912b95f51e6b2d951cab2e424f98ec73158d4ab66699fe53cb11ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-pj1rhy8i/wheels/bd/2e/c1/27649c1e84270617e21539a0da419b3f469ace0b6b9ee26356
Successfully built deeppavlov
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-t6b3owqk
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-t6b3

In [ ]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu_with_features
!pip install git+https://github.com/deepmipt/deeppavlov.git@bert_sequence_multiinput
!pip install tensorflow==1.14.0
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.4

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu_with_features) to /tmp/pip-req-build-yo7lhzdo
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-yo7lhzdo
  Running command git checkout -b feat/multi_gpu_with_features --track origin/feat/multi_gpu_with_features
  Switched to a new branch 'feat/multi_gpu_with_features'
  Branch 'feat/multi_gpu_with_features' set up to track remote branch 'feat/multi_gpu_with_features' from 'origin'.
  Created wheel for bert-dp: filename=bert_dp-1.0-cp36-none-any.whl size=23630 sha256=693f7fbc49e02680a317aa71ca1834dabebf6cd58ae7a899ac9c06d255efcc19
  Stored in directory: /tmp/pip-ephem-wheel-cache-oqb07pqh/wheels/9e/61/2d/d233500d0332b45f99ec641661d32dbcc6303eb07a4790bbd8
Successfully built bert-dp
  Cloning https://github.com/deepmipt/deeppavlov.git (to revision bert_sequence_multiinput) to /tmp/pip-req-build-vzh4k1wv
  Running command git clone -q https://github.com/deepmipt/deeppavlov.git /

     |████████████████████████████████| 109.2MB 38kB/s 
     |████████████████████████████████| 491kB 53.9MB/s 
     |████████████████████████████████| 3.2MB 41.9MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 377.0MB 33kB/s 
     |████████████████████████████████| 317kB 65kB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
from deeppavlov import build_model, train_evaluate_model_from_config, train_model

basic = train_model('/content/drive/My Drive/terms/files/simple/config_basic.json', download=True)
#model = train_model('/content/drive/My Drive/terms/config1.json')

2020-06-07 12:11:12.899 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt
347MB [00:43, 8.07MB/s]
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-06-07 12:11:58.380 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/drive/My Drive/terms/files/simple/word.dict]
2020-06-07 12:11:58.524 INFO in 'deeppavlov.core.data.simple_vocab'['simple_








Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

Instructions for updating:
Use `tf.cast` instead.


2020-06-07 12:12:43.807 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


...

2020-06-07 12:22:27.812 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/simple/word.dict]
2020-06-07 12:22:27.829 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/simple/tag.dict]
2020-06-07 12:22:27.835 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/simple/char.dict]
2020-06-07 12:22:27.838 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-op

2020-06-07 12:23:09.537 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


2020-06-07 12:23:09.736 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


2020-06-07 12:23:12.342 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/terms/files/simple/model_simple]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/terms/files/simple/model_simple


In [ ]:
from deeppavlov import build_model, train_evaluate_model_from_config, train_model

basic_pos = train_model('/content/drive/My Drive/terms/files/gaz_pos/config_basic_pos.json', download=True)

2020-06-07 12:58:28.373 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt download because of matching hashes
2020-06-07 12:58:28.698 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz_pos/word.dict]
2020-06-07 12:58:28.877 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/drive/My Drive/terms/files/gaz_pos/word.dict]
2020-06-07 12:58:28.887 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz_pos/pos_1.dict]
2020-06-07 12:58:28.896 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/drive/My Drive/terms/files/gaz_pos/pos_1.dict]
2020-06-07 12:58:28.905 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary 

2020-06-07 12:59:11.327 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


2020-06-07 12:59:11.527 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


UnboundLocalError: ignored

In [ ]:
from deeppavlov import build_model, train_evaluate_model_from_config, train_model

bert_pos = train_model('/content/drive/My Drive/terms/files/gaz_pos/bert/config_bert_pos.json', download=True)

2020-06-07 20:31:24.785 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/cased_L-12_H-768_A-12.zip download because of matching hashes
2020-06-07 20:31:24.919 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2020-06-07 20:31:25.16 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz_pos/tag.dict]
2020-06-07 20:31:25.63 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/drive/My Drive/terms/files/gaz_pos/tag.dict]
2020-06-07 20:31:25.71 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz_pos/pos.dict]
2020-06-07 20:31:25.102 INFO in 'deeppavlov.core.data.simple_vocab'['simple_

2020-06-07 20:31:59.967 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/terms/files/gaz_pos/model_bert_pos]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/terms/files/gaz_pos/model_bert_pos


2020-06-07 20:33:06.220 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 11160 tokens with 1060 phrases; found: 1096 phrases; correct: 0.

precision:  54.56%; recall:  56.42%; FB1:  55.47

	Material: precision:  65.58%; recall:  69.43%; F1:  67.45 523

	Process: precision:  48.45%; recall:  51.05%; F1:  49.72 452

	Task: precision:  29.75%; recall:  26.28%; F1:  27.91 121


2020-06-07 20:33:06.233 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 201: Initial best ner_f1 of 55.4731


{"valid": {"eval_examples_count": 413, "metrics": {"ner_f1": 55.4731, "ner_token_f1": 62.4935}, "time_spent": "0:01:03", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

2020-06-07 20:58:45.387 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 21711 tokens with 1892 phrases; found: 2010 phrases; correct: 0.

precision:  43.63%; recall:  46.35%; FB1:  44.95

	Material: precision:  48.09%; recall:  55.00%; F1:  51.31 915

	Process: precision:  44.28%; recall:  45.61%; F1:  44.93 926

	Task: precision:  15.98%; recall:  13.99%; F1:  14.92 169




{"test": {"eval_examples_count": 829, "metrics": {"ner_f1": 44.9513, "ner_token_f1": 50.26}, "time_spent": "0:02:05"}}


2020-06-07 20:58:45.717 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz_pos/tag.dict]
2020-06-07 20:58:45.722 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz_pos/pos.dict]


2020-06-07 20:59:18.47 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/terms/files/gaz_pos/model_bert_pos]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/terms/files/gaz_pos/model_bert_pos


In [ ]:
from deeppavlov import build_model, train_evaluate_model_from_config, train_model

bert_gaz = train_model('/content/drive/My Drive/terms/files/gaz/bert/config_bert_vector.json', download=True)

2020-06-07 18:30:15.195 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/cased_L-12_H-768_A-12.zip download because of matching hashes
2020-06-07 18:30:15.445 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2020-06-07 18:30:15.540 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz/bert/tag.dict]
2020-06-07 18:30:15.596 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/drive/My Drive/terms/files/gaz/bert/tag.dict]


2020-06-07 18:30:48.832 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/terms/files/gaz/bert/model_bert_gaz]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/terms/files/gaz/bert/model_bert_gaz


2020-06-07 18:32:10.949 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 11160 tokens with 1060 phrases; found: 1073 phrases; correct: 0.

precision:  56.01%; recall:  56.70%; FB1:  56.35

	Material: precision:  66.21%; recall:  67.81%; F1:  67.00 506

	Process: precision:  51.45%; recall:  53.61%; F1:  52.51 447

	Task: precision:  30.00%; recall:  26.28%; F1:  28.02 120


2020-06-07 18:32:10.962 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 201: Initial best ner_f1 of 56.3526


{"valid": {"eval_examples_count": 413, "metrics": {"ner_f1": 56.3526, "ner_token_f1": 61.1333}, "time_spent": "0:01:18", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

2020-06-07 19:43:32.335 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 21711 tokens with 1892 phrases; found: 1997 phrases; correct: 0.

precision:  43.62%; recall:  46.04%; FB1:  44.79

	Material: precision:  49.17%; recall:  55.62%; F1:  52.20 905

	Process: precision:  44.21%; recall:  44.16%; F1:  44.18 898

	Task: precision:  14.95%; recall:  15.03%; F1:  14.99 194




{"test": {"eval_examples_count": 829, "metrics": {"ner_f1": 44.793, "ner_token_f1": 49.0373}, "time_spent": "0:02:05"}}


2020-06-07 19:43:32.760 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/My Drive/terms/files/gaz/bert/tag.dict]


2020-06-07 19:44:04.241 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/terms/files/gaz/bert/model_bert_gaz]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/terms/files/gaz/bert/model_bert_gaz
